# Get gender

## Add `'gender'` key to database

In [1]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
import json

In [2]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [3]:
D=gender.Detector(case_sensitive=False)

In [4]:
def force_get_gender(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = ''
    if (d.get('name') and d.get('name').get('value') and 
        sub('\w\.','',d.get('name').get('value').split(', ')[-1]).strip()):
        names = sub('\w\.','',d.get('name').get('value').split(', ')[-1]).strip()
        names = names.replace('-',' ').strip()
    if not names and d.get('full_name'):
        names = sub('\w\.','',d.get('full_name').split(', ')[-1]).strip()
        names = names.replace('-',' ').strip()
    if names:
        if d.get('country'):
            gender = D.get_gender( names, country=d.get('country').strip() )
        else:
            gender = D.get_gender( names )
    return gender

In [5]:
kk = [d.update( {'gender': force_get_gender(d)} ) for d in df ]

In [6]:
f=open('inspire_LA_gender.json','w')
json.dump(df,f)
f.close()

## Analysis

In [7]:
import pandas as pd

In [8]:
DF = pd.read_json('inspire_LA_gender.json')

Unique authors ids

In [9]:
DF.shape[0]

57588

In [10]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

29969

In [11]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [12]:
DF['first_names'] = DF.apply(lambda row: str(row['full_name']).split(', ')[-1]  
                             if row['first_names']=='None' 
                             else row['first_names'] ,axis = 'columns')

In [13]:
DF['gender'].unique()

array(['male', 'unknown', 'female', 'andy'], dtype=object)

In [14]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

29969


29969

In [15]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [16]:
assert male + female + unknown == total

In [17]:
male+female

17491

In [18]:
male

14755

In [19]:
female

2736

In [20]:
round(female/male,2)

0.19

In [21]:
unknown

12478

In [22]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

(17491, 14)

In [23]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

(12478, 14)

In [28]:
DFFM[['author_id','first_names','country','gender']].sample(50)

,author_id,first_names,country,gender
9696,V.Petracek.1,Vojtěch,Czechia,male
7031,M.De.Francia.1,Marcelo,Argentina,male
29659,H.Stoeckinger.Kim.2,Hyejung,Germany,female
12184,S.Benic.1,Sanjin,Croatia,male
9769,L.B.Auerbach.1,Leonard,United States,male
26535,P.S.Julienne.1,Paul S.,United States,male
12799,R.Guardiola.1,Rafael,Spain,male
23493,R.G.Edwards.1,Robert Glenn,United States,male
3341,G.Preparata.1,Giuliano,Italy,male
8679,M.Le.Delliou.1,Morgan,Brazil,male


In [25]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

,author_id,full_name,name,country,gender
22752,F.da.Silva.Bortoli.1,"da Silva Bortoli, F.",None,Brazil,unknown
29221,E.S.da.Fonseca.1,"da Fonseca, E.S.",None,Brazil,unknown
10686,M.Miller.bertolami.1,"Miller Bertolami, M.",None,Argentina,unknown
7465,S.Gliske.1,"Gliske, S.",None,United States,unknown
10666,A.de.la.Torre.1,"de la Torre, A.C.",None,Argentina,unknown
3118,G.C.Orozco.1,"Orozco, G.C.",None,Colombia,unknown
11182,D.Pohlblanc.1,"Pohlblanc, D.",None,France,unknown
21505,C.P.Ferreira.2,"Ferreira, C.P.",None,Brazil,unknown
4785,M.Correnti.1,"Correnti, M.",None,Italy,unknown
6252,J.L.Pineda.1,"Pineda, J.L.",None,United States,unknown


check specific entries

In [30]:
JSON([d for d in df if d.get('author_id')=='F.Fontanot.3'][0])

<IPython.core.display.JSON object>